In [50]:
import nltk
from typing import Dict, List, Optional, Tuple
import json
import os
from indexingcode.utils.Preprocessor import Preprocessor
import ijson
from tqdm import tqdm
import gensim.utils

nltk.download('all', quiet=True)

True

In [51]:
data_path = '/Users/wojciechsitek/Local/Indexing/data/BioAsq2021/allMeSH_2021.json'

preprocessor = Preprocessor(gensim.utils.tokenize, nltk.PorterStemmer())



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/wojciechsitek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:
with open('/Users/wojciechsitek/Local/Indexing/data/BioAsq-test1/bioasq_test_50tys.json') as f:
    objects = ijson.items(f, 'articles.item')
    i = 0
    j = 0
    for a in tqdm(objects):
        i += 1
    f.seek(0)
    for b in tqdm(objects):
        j += 1
    print(i)

20it [00:00, 24364.24it/s]
0it [00:00, ?it/s]


IncompleteJSONError: parse error: trailing garbage
                                       {"articles":[ {"journal":"The j
                     (right here) ------^


In [53]:
# abstracts
class AbstractIterator:
    def __init__(self, f):
        self.f = f

    def __iter__(self):
        self.f.seek(0)
        for article in tqdm(ijson.items(self.f, 'articles.item'), total=15559157):
            yield preprocessor.preprocess_text(article['abstractText'])

In [54]:
import gensim

out_dir = '/Users/wojciechsitek/Documents/Indexing/code/indexingcode/out/article-preprocess-2021.nosync'
# with open(data_path) as f:
with open('/Users/wojciechsitek/Local/Indexing/data/BioAsq2021/allMeSH_2021.json', encoding='ISO-8859-1') as f:
    model = gensim.models.Word2Vec(vector_size=1, window=5, min_count=2, epochs=2, sg=1)
    sentences_iterator = AbstractIterator(f)
    print("Build vocab")
    model.build_vocab(sentences_iterator)
    print("Train")
    model.train(sentences_iterator, total_examples=model.corpus_count, epochs=model.epochs)
    print("Build vocab dictionaries")

    num_to_emb = dict(map(lambda i: (i, model.wv[i][0]), filter(lambda word: word in model.wv.key_to_index, preprocessor.vocabulary.values())))
    num_to_text = dict(map(lambda a: (a[1], a[0]), preprocessor.vocabulary.items()))
    print("Build abstracts out")
    def iter_abstracts_out():
        f.seek(0)
        for article in ijson.items(f, 'articles.item'):
            yield ([num_to_emb[word] for word in filter(lambda word: word in model.wv.key_to_index, article)], preprocessor.preprocess_journal(article['journal']), preprocessor.preprocess_mesh(article['meshMajor']))
    with open(os.path.join(out_dir,'bioasq_preprocessed_all.csv'), 'w') as fw:
        for line, journal, meshes in tqdm(iter_abstracts_out(), total=15559157):
            fw.write(f"{str(journal)};{','.join(map(str, line))};{','.join(map(str, meshes))}\n")
    with open(os.path.join(out_dir, 'num_to_emb.json'), 'w') as fw:
        json.dump([(str(k),str(v)) for k, v in num_to_emb.items()], fw)

    with open(os.path.join(out_dir, 'num_to_text.json'), 'w') as fw:
        json.dump([(str(k),str(v)) for k, v in num_to_text.items()], fw)

    with open(os.path.join(out_dir, 'mesh_vocabulary.json'), 'w') as fw:
        json.dump([(str(k),str(v)) for k, v in preprocessor.mesh_vocabulary.items()], fw)


Build vocab


  1%|▏         | 225099/15559157 [15:01<17:03:23, 249.73it/s]


KeyboardInterrupt: 

In [39]:
type(iter_abstracts_out())

generator

In [ ]:
import json


In [ ]:
# TEST FOR OUT LINE
print([(a, num_to_text[a], num_to_emb[a]) for a in range(1, 5)])
list(map(lambda article: list(map(lambda i: num_to_emb[i], preprocessor.preprocess_text(article['abstractText']))), [{'abstractText':'traumat brain'}, {'abstractText': 'injuri tbi'}]))

In [7]:
# with open('/Users/wojciechsitek/Local/Indexing/out/article-preprocess/bioasq_preprocessed.csv', 'w') as f:
#     for line in abstracts_out:
#         f.write(f"{','.join(map(str, line))}\n")

In [ ]:
import json
with open(os.path.join(out_dir, 'mesh_vocabulary.json'), 'w') as f:
    json.dump([(str(k),str(v)) for k, v in preprocessor.mesh_vocabulary.items()], f)

In [15]:
del abstracts_out
del journals_out
del meshes_out

In [130]:
num_to_emb = {}
num_to_text = {}
for word, i in preprocessor.vocabulary.items():
    num_to_emb[i] = model.wv[i][0]
    num_to_text[i] = word

In [124]:
num_to_emb_sorted = [(k, v, num_to_text[k]) for k, v in sorted(num_to_emb.items(), key=lambda a: a[1])]

In [126]:
num_to_emb_sorted[1000:]

[(19494, -1.9563501, 'citric'),
 (4901, -1.9563332, 'herbivor'),
 (24089, -1.9562715, 'adma'),
 (3580, -1.9562068, 'chd'),
 (3776, -1.956191, 'cure'),
 (594, -1.9561521, 'daytim'),
 (1869, -1.955883, 'radiu'),
 (8539, -1.9557395, 'underscor'),
 (8738, -1.9557378, 'modest'),
 (11246, -1.955703, 'aecopd'),
 (2376, -1.9556928, 'suit'),
 (589, -1.9555063, 'anxieti'),
 (9586, -1.9554634, 'sigmoid'),
 (16728, -1.9553767, 'subdivid'),
 (8584, -1.9551866, 'oocyt'),
 (4925, -1.9550589, 'undisturb'),
 (27446, -1.9550502, 'stsm'),
 (5652, -1.954813, 'terminolog'),
 (10485, -1.9548012, 'ut'),
 (16893, -1.9547744, 'corrobor'),
 (6821, -1.9547523, 'mycoplasma'),
 (4130, -1.9545218, 'reoper'),
 (22490, -1.9543761, 'partum'),
 (15235, -1.954339, 'bortezomib'),
 (18306, -1.9541602, 'transderm'),
 (5617, -1.9540714, 'counsel'),
 (13239, -1.9538181, 'assert'),
 (420, -1.95378, 'weekli'),
 (13316, -1.9537363, 'abbottabad'),
 (1251, -1.9536935, 'unifi'),
 (10199, -1.9536885, 'morn'),
 (9918, -1.9536754, 'f

In [16]:
preprocessor.vocabulary

{'background': 0,
 'traumat': 1,
 'brain': 2,
 'injuri': 3,
 'tbi': 4,
 'lead': 5,
 'caus': 6,
 'death': 7,
 'disabl': 8,
 'worldwid': 9,
 'examin': 10,
 'nationwid': 11,
 'trend': 12,
 'relat': 13,
 'hospit': 14,
 'mortal': 15,
 'april': 16,
 'march': 17,
 'use': 18,
 'popul': 19,
 'base': 20,
 'databas': 21,
 'mandatori': 22,
 'canada': 23,
 'method': 24,
 'rate': 25,
 'acut': 26,
 'separ': 27,
 'analyz': 28,
 'linear': 29,
 'regress': 30,
 'independ': 31,
 'predictor': 32,
 'evalu': 33,
 'logist': 34,
 'result': 35,
 'remain': 36,
 'stabl': 37,
 'children': 38,
 'young': 39,
 'adult': 40,
 'increas': 41,
 'consider': 42,
 'among': 43,
 'elderli': 44,
 'age': 45,
 'q': 46,
 'year': 47,
 'fall': 48,
 'motor': 49,
 'vehicl': 50,
 'collis': 51,
 'mvc': 52,
 'common': 53,
 'p': 54,
 'decreas': 55,
 'vulner': 56,
 'experienc': 57,
 'greatest': 58,
 'risk': 59,
 'declin': 60,
 'admiss': 61,
 'signific': 62,
 'toward': 63,
 'sever': 64,
 'comorbid': 65,
 'length': 66,
 'stay': 67,
 'howev':